In [1]:
# Import parquet file with pySpark
from pyspark.sql import SparkSession

# Create a spark session
spark = SparkSession.builder.appName("parquet").getOrCreate()

# Read parquet file
df = spark.read.parquet("ml_data_train.parquet")

# Show the data
df.show()


+------------------+--------------+--------------------------+--------------------+--------------------------+-----------------------+-----------------------+------------+
| src_subnet_hashed|dst_subnet_vec|scaled_bytes_pkts_features|scaled_flow_duration|scaled_throughput_features|scaled_pkt_len_features|scaled_tcp_win_features|attack_index|
+------------------+--------------+--------------------------+--------------------+--------------------------+-----------------------+-----------------------+------------+
|(1024,[906],[1.0])|(94,[0],[1.0])|      [-0.0513064471342...|[-0.4789529772200...|      [-0.0125662332574...|   [-0.4592399504915...|   [-0.4787111328395...|         1.0|
|(1024,[906],[1.0])|(94,[0],[1.0])|      [-0.0513064471342...|[-0.4789529772200...|      [-0.0125662332574...|   [-0.4592399504915...|   [-0.4787111328395...|         1.0|
|(1024,[906],[1.0])|(94,[0],[1.0])|      [-0.0517375603737...|[-0.4789529772200...|      [-0.0125662862983...|   [-0.4676085019167...|   [-0

#### Feature Scalling
Feature scaling can play a crucial role in optimizing the performance of machine learning models, especially those sensitive to the scale of input data.
Applying feature scaling to the right variables essential for improving model accuracy and efficiency. It ensures that each feature contributes equally to the decision-making process, preventing models from misinterpreting the data due to arbitrary feature scales. This leads to better, more reliable predictions in multiclass classification tasks.

##### Numerical Variables with Different Scales:

In [ ]:
# Statistics summary for numerical variables candidates for scalling
columns_of_interest_for_scaling = [
    "IN_BYTES", "OUT_BYTES", "IN_PKTS", "OUT_PKTS", 
    "FLOW_DURATION_MILLISECONDS", "SRC_TO_DST_SECOND_BYTES", 
    "DST_TO_SRC_SECOND_BYTES", "LONGEST_FLOW_PKT", "SHORTEST_FLOW_PKT", 
    "MIN_IP_PKT_LEN", "MAX_IP_PKT_LEN", "TCP_WIN_MAX_IN", "TCP_WIN_MAX_OUT"
]

FEATURES = [
    "L4_SRC_PORT",
    "L4_DST_PORT",
    #"ICMP_TYPE",
    #"ICMP_IPV4_TYPE",
    #"DNS_QUERY_ID",
    #"DNS_QUERY_TYPE",
    #"DNS_TTL_ANSWER",
    #"FTP_COMMAND_RET_CODE",
    #"IN_BYTES",
    #"OUT_BYTES",
    #"IN_PKTS",
    #"OUT_PKTS",
    #"FLOW_DURATION_MILLISECONDS",
    #"PROTOCOL",
    #"SRC_TO_DST_SECOND_BYTES",
    #"DST_TO_SRC_SECOND_BYTES",
    #"LONGEST_FLOW_PKT",
    #"SHORTEST_FLOW_PKT",
    #"MIN_IP_PKT_LEN",
    #"MAX_IP_PKT_LEN",
    #"TCP_WIN_MAX_IN",
    #"TCP_WIN_MAX_OUT",
    #"dst_subnet_index",
    #"src_subnet_index",
    #"CLIENT_TCP_FLAGS",
    #"SERVER_TCP_FLAGS",
    #"DURATION_OUT",
    #"DURATION_IN",
    #"MIN_TTL",
    "NUM_PKTS_UP_TO_128_BYTES",
    #"RETRANSMITTED_IN_BYTES",
    #"MAX_TTL",
    "NUM_PKTS_128_TO_256_BYTES",
    #"DST_TO_SRC_AVG_THROUGHPUT",
    "NUM_PKTS_512_TO_1024_BYTES",
    "NUM_PKTS_256_TO_512_BYTES",
    #"RETRANSMITTED_OUT_BYTES",
    #"RETRANSMITTED_OUT_PKTS",
    #"RETRANSMITTED_IN_PKTS",
    #"SRC_TO_DST_AVG_THROUGHPUT",
    #"L7_PROTO",
    "NUM_PKTS_1024_TO_1514_BYTES",
    #"TCP_FLAGS",
]

In [ ]:
from typing import Protocol
from pyspark.ml.feature import VectorAssembler, StandardScaler, FeatureHasher, OneHotEncoder
from pyspark.ml import Pipeline

# OneHot encoding for destination subnet since higher cardinality
dst_subnet_encoder = OneHotEncoder(inputCol="dst_subnet_index", outputCol="dst_subnet_vec")

# Source address
# Feature hashing for source subnet since lower cardinality
src_subnet_hasher = FeatureHasher(inputCols=["src_subnet_index"], outputCol="src_subnet_hashed", numFeatures=1024)

# Assemble numerical features for bytes and packets trasmitted
trasimitted_in_features = ["IN_BYTES", "IN_PKTS"]
trasmitted_out_features = ["OUT_BYTES", "OUT_PKTS"]
retransimtted_in_features = ["RETRANSMITTED_IN_BYTES", "RETRANSMITTED_IN_PKTS"]
retransimtted_out_features = ["RETRANSMITTED_OUT_BYTES", "RETRANSMITTED_OUT_PKTS"]
throughput_in_features = ["SRC_TO_DST_SECOND_BYTES", "SRC_TO_DST_AVG_THROUGHPUT"]
throughput_out_features = ["DST_TO_SRC_AVG_THROUGHPUT", "DST_TO_SRC_SECOND_BYTES"]
ttl_features = ["MIN_TTL", "MAX_TTL"]
tcp_flags_features = ["CLIENT_TCP_FLAGS", "SERVER_TCP_FLAGS", "TCP_FLAGS"]
icmp_features = ["ICMP_TYPE", "ICMP_IPV4_TYPE"]
flow_max_features = ["LONGEST_FLOW_PKT", "MAX_IP_PKT_LEN"]
flow_min_features = ["SHORTEST_FLOW_PKT", "MIN_IP_PKT_LEN"]
flow_duration_features = ["FLOW_DURATION_MILLISECONDS"]
tcp_in_features = ["TCP_WIN_MAX_IN"]
tcp_out_features = ["TCP_WIN_MAX_OUT"]
duration_in_features = ["DURATION_IN"]
duration_out_features = ["DURATION_OUT"]
protocol_features = ["PROTOCOL"]
l4_src_port_features = ["L4_SRC_PORT"]
l4_dst_port_features = ["L4_DST_PORT"]
dns_features = ["DNS_QUERY_ID", "DNS_QUERY_TYPE", "DNS_TTL_ANSWER"]
ftp_features = ["FTP_COMMAND_RET_CODE"]
l7_proto_features = ["L7_PROTO"]
num_packets_by_size_features = ["NUM_PKTS_UP_TO_128_BYTES", "NUM_PKTS_128_TO_256_BYTES", "NUM_PKTS_256_TO_512_BYTES", "NUM_PKTS_512_TO_1024_BYTES", "NUM_PKTS_1024_TO_1514_BYTES"]



# Assemble numerical features for flow duration
flow_duration_assembler = VectorAssembler(
    inputCols=["FLOW_DURATION_MILLISECONDS"],
    outputCol="flow_duration_feature"
)

tcp__assembler = VectorAssembler(
    inputCols=["TCP_WIN_MAX_IN", "TCP_WIN_MAX_OUT"],
    outputCol="tcp_win_features",
)

# Apply StandardScaler to the assembled vectors
bytes_pkts_scaler = StandardScaler(
    inputCol="bytes_pkts_features", 
    outputCol="scaled_bytes_pkts_features", 
    withStd=True, 
    withMean=True
)

flow_duration_scaler = StandardScaler(
    inputCol="flow_duration_feature", 
    outputCol="scaled_flow_duration", 
    withStd=True, 
    withMean=True
)

throughput_scaler = StandardScaler(
    inputCol="throughput_features", 
    outputCol="scaled_throughput_features", 
    withStd=True, 
    withMean=True
)

pkt_len_scaler = StandardScaler(
    inputCol="pkt_len_features", 
    outputCol="scaled_pkt_len_features", 
    withStd=True, 
    withMean=True
)

tcp_win_scaler = StandardScaler(
    inputCol="tcp_win_features", 
    outputCol="scaled_tcp_win_features", 
    withStd=True, 
    withMean=True
)

# Create a pipeline for scaling
scaling_pipeline = Pipeline(stages=[
    dst_subnet_encoder,
    src_subnet_hasher,
    bytes_pkts_assembler, 
    bytes_pkts_scaler, 
    flow_duration_assembler, 
    flow_duration_scaler, 
    throughput_assembler, 
    throughput_scaler, 
    pkt_len_assembler, 
    pkt_len_scaler, 
    tcp_win_assembler, 
    tcp_win_scaler
])

# Fit the scalling pipeline to the encoded training data
scalling_model = scaling_pipeline.fit(encoded_train_data)

# Transform both encoded training and test data
processed_train_data = scalling_model.transform(encoded_train_data)
processed_test_data = scalling_model.transform(encoded_test_data)

In [2]:
from pyspark.ml.feature import VectorAssembler

# Preparar os dados com VectorAssembler
feature_columns = ['scaled_bytes_pkts_features', 'scaled_flow_duration', 'scaled_throughput_features', 'scaled_pkt_len_features', 'scaled_tcp_win_features']
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df_assembled = assembler.transform(df)

In [3]:
from pyspark.ml.classification import RandomForestClassifier, NaiveBayes, LogisticRegression, MultilayerPerceptronClassifier

# Definir o Random Forest
rf = RandomForestClassifier(labelCol="attack_index", featuresCol="features", numTrees=10)

# Definir o Naive Bayes
#nb = NaiveBayes(labelCol="attack_index", featuresCol="features")

# Definir o Logistic Regression
lr = LogisticRegression(labelCol="attack_index", featuresCol="features")

# Configuração do Multilayer Perceptron
# Obter o número de características
def get_feature_count(df, feature_col="features"):
    # Extrai os metadados da coluna de características e calcula a soma dos tamanhos dos atributos
    attributes = df.schema[feature_col].metadata["ml_attr"]["attrs"]
    feature_count = sum(len(attrs) for attrs in attributes.values())
    return feature_count
input_layers = get_feature_count(df_assembled, "features")
output_layers = df_assembled.select("attack_index").distinct().count()
hidden_layers = [input_layers, (input_layers + output_layers) // 2, output_layers]
mlp = MultilayerPerceptronClassifier(labelCol="attack_index", featuresCol="features", layers=hidden_layers, maxIter=100)

In [4]:
# Treinamento dos modelos
model_rf = rf.fit(df_assembled)
#model_nb = nb.fit(df_assembled)
model_lr = lr.fit(df_assembled)
model_mlp = mlp.fit(df_assembled)

In [5]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Avaliador
evaluator = MulticlassClassificationEvaluator(labelCol="attack_index", predictionCol="prediction", metricName="accuracy")

# Avaliar Random Forest
accuracy_rf = evaluator.evaluate(model_rf.transform(df_assembled))
print(f"Random Forest Accuracy: {accuracy_rf}")

# Avaliar Naive Bayes
#accuracy_nb = evaluator.evaluate(model_nb.transform(df_assembled))
#print(f"Naive Bayes Accuracy: {accuracy_nb}")

# Avaliar Logistic Regression
accuracy_lr = evaluator.evaluate(model_lr.transform(df_assembled))
print(f"Logistic Regression Accuracy: {accuracy_lr}")

# Avaliar Multilayer Perceptron
accuracy_mlp = evaluator.evaluate(model_mlp.transform(df_assembled))
print(f"Multilayer Perceptron Accuracy: {accuracy_mlp}")

Random Forest Accuracy: 0.8129
Logistic Regression Accuracy: 0.6209
Multilayer Perceptron Accuracy: 0.8125
